In [2]:
import pandas as pd
import parse_functions as pf

# Vorbereitung: Einlesen der Daten
- Schwierigkeit: Größe des Datensatzes: 10.365.152 Daten (über 10 Milionen)
- keine eindeutiges Trennzeichen in den Daten vorhanden
- kein von Pandas vorgefertigter Import für Log Dateien

In [3]:
# Source: https://mmas.github.io/read-apache-access-log-pandas
access_log = pd.read_csv(
    'data/access.log',
    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\]\s)',
    engine='python',
    na_values='-',
    header=None,
    usecols=[0, 3, 4, 5, 6, 7, 8],
    names=['ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'],
    converters={'time': pf.parse_datetime,
                'request': pf.parse_str,
                'status': int,
                'size': int,
                'referer': pf.parse_str,
                'user_agent': pf.parse_str},
    on_bad_lines='warn')

access_log.head()


,ip,time,request,status,size,referer,user_agent
0,54.36.149.41,2019-01-22 03:56:14+03:30,GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%D...,200,30577,NaN,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...
1,31.56.96.51,2019-01-22 03:56:16+03:30,GET /image/60844/productModel/200x200 HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
2,31.56.96.51,2019-01-22 03:56:16+03:30,GET /image/61474/productModel/200x200 HTTP/1.1,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
3,40.77.167.129,2019-01-22 03:56:17+03:30,GET /image/14925/productModel/100x100 HTTP/1.1,200,1696,NaN,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
4,91.99.72.15,2019-01-22 03:56:17+03:30,GET /product/31893/62100/%D8%B3%D8%B4%D9%88%D8...,200,41483,NaN,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...


## Umwandlung in CSV Datei
- Damit nicht jedes Mal Daten neu über das Log eingelesen werden müssen!
- CSV Datei erstellen und für erneutes einlesen nutzbar (geht schneller)

In [4]:
#Daten in einer CSV Datei speicher
filename = "data/acces_log.csv"
access_log.to_csv(filename)

In [5]:
# Daten aus einer CSV Date einladen
filename = "data/acces_log.csv"
access_log_csv = pd.read_csv(filename)
len(access_log_csv)

10365152

In [6]:
#Erzeuge eine Kopie der Daten als "Backup"
access_log_backup = access_log.copy()
len(access_log_backup)

10365152

# Aufgabe 1: Beliebtestes Produkt

> Analysieren Sie welche Produkte beliebt sind. Entwickeln Sie dazu eine Definition eines beliebten Produktes. Stellen Sie die Ergebnisse anschaulich da.

## Definition

> **Unsere Definition:**  
  Das Produkt mit dein meisten Aufrufen auf dem Webserver

## Ergebnis


In [8]:
most_viewed_products = access_log.loc[access_log['request'].str.contains(r'^GET /product/\d+', na=False)].value_counts(access_log['request'])
print(most_viewed_products[0], most_viewed_products.keys()[0])

453 GET /product/33968/64756/%DA%AF%D9%88%D8%B4%DB%8C-%D9%85%D9%88%D8%A8%D8%A7%DB%8C%D9%84-%D8%B3%D8%A7%D9%85%D8%B3%D9%88%D9%86%DA%AF-%D9%85%D8%AF%D9%84-Galaxy-J6-Plus-%282018%29-Dual-32GB-%28J610F-DS%29 HTTP/1.1


# Aufgabe 2

> Untersuchen Sie den Datensatz auf weitere Auffälligkeiten.


## Basisinformationen durch Panda Befehle

### Befehl .info()


In [11]:
access_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10365152 entries, 0 to 10365151
Data columns (total 7 columns):
 #   Column      Dtype                                
---  ------      -----                                
 0   ip          object                               
 1   time        datetime64[ns, pytz.FixedOffset(210)]
 2   request     object                               
 3   status      int64                                
 4   size        int64                                
 5   referer     object                               
 6   user_agent  object                               
dtypes: datetime64[ns, pytz.FixedOffset(210)](1), int64(2), object(4)
memory usage: 553.6+ MB


### Befehl .describe()
- Anzahl der Einträge
- wie viele einzigartige Einträge
- höchste Zahl
- ...

--> Statistische Basisinformationen

In [10]:
access_log.describe(include = 'all')

,ip,time,request,status,size,referer,user_agent
count,10365152,10365152,10365146,1.036515e+07,1.036515e+07,8718773,10350668
unique,258606,NaN,894210,NaN,NaN,103236,28342
top,66.249.66.194,NaN,GET /settings/logo HTTP/1.1,NaN,NaN,https://www.zanbil.ir/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
freq,353483,NaN,352037,NaN,NaN,494757,746572
mean,NaN,2019-01-24 13:02:56.248167680+03:30,NaN,2.101419e+02,1.243311e+04,NaN,NaN
min,NaN,2019-01-22 03:56:14+03:30,NaN,2.000000e+02,0.000000e+00,NaN,NaN
25%,NaN,2019-01-23 09:32:56+03:30,NaN,2.000000e+02,2.027000e+03,NaN,NaN
50%,NaN,2019-01-24 12:09:06+03:30,NaN,2.000000e+02,4.120000e+03,NaN,NaN
75%,NaN,2019-01-25 20:22:33+03:30,NaN,2.000000e+02,1.155100e+04,NaN,NaN
max,NaN,2019-01-26 20:29:13+03:30,NaN,5.040000e+02,1.249490e+06,NaN,NaN


In [9]:
# IP-Adressen mit den meisten Aufrufen 
access_log.value_counts("ip")

ip
66.249.66.194      353483
66.249.66.91       314522
151.239.241.163     92475
66.249.66.92        88332
91.99.30.32         45979
                    ...  
5.217.114.109           1
2.185.112.228           1
37.44.58.168            1
5.119.191.23            1
5.52.183.148            1
Name: count, Length: 258606, dtype: int64

In [ ]:
access_log['time']